In [ ]:
# default_exp cameras

# OpenHSI Camera Implementations

> Hardware cameras

Wrapper class and example code for getting images from the OpenHSI using a ximea detetor (with IMX252 sensor, e.g. [MX031CG-SY](https://www.ximea.com/en/products/xilab-application-specific-custom-oem/embedded-vision-and-multi-camera-setup-xix/sony-imx252-fast-color-industrial-camera)). 



In [ ]:
#hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# imitation of Julia's multiple dispatch using @typedispatch
from fastcore.dispatch import typedispatch

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates

In [ ]:
#export 

from fastcore.foundation import patch
from fastcore.meta import delegates
import cv2
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm

In [ ]:
#export
from openhsi.capture import OpenHSI

In [ ]:
# export

@delegates()
class WebCamera(OpenHSI):
    """Interface for webcam to test OpenHSI functionality"""
    def __init__(self, mode:str = None, **kwargs):
        """Initialise Webcam"""
        super().__init__(**kwargs)
        self.mode = mode
        if self.mode == "HgAr":
            self.gen = self.gen_sim_spectra()
        
        # Check if the webcam is opened correctly
        self.vid = cv2.VideoCapture(0)
        if not self.vid.isOpened():
            raise IOError("Cannot open webcam")
    
    def start_cam(self):
        pass
    
    def stop_cam(self):
        self.vid.release()
        cv2.destroyAllWindows()
    
    def get_img(self) -> np.ndarray:
        if self.mode == "HgAr":
            return next(self.gen)
        else:
            ret, frame = self.vid.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            frame = cv2.resize(frame, tuple(np.flip(self.settings["resolution"])), interpolation = cv2.INTER_AREA)
            if self.mode == "crop":
                frame[:self.settings["row_slice"][0],...] = 0
                frame[self.settings["row_slice"][1]:,...] = 0
            return frame
    
    def gen_sim_spectra(self):
        lines_nm = [254,436,546,764,405,365,578,750,738,697,812,772,912,801,842,795,706,826,852,727] # approx sorted by emission strength
        img = np.zeros(tuple(self.settings["resolution"]),dtype=np.uint8)
        wavelengths = np.linspace(self.settings["index2wavelength_range"][0],self.settings["index2wavelength_range"][1],num=self.settings["resolution"][1])

        strength = 255
        for line in lines_nm: 
            indx = np.sum(wavelengths<line)
            if indx > 0 and indx < self.settings["resolution"][1]:
                img[:,indx-2:indx+2] = strength
                strength -= 5
        while True:
            yield img
    
    def get_temp(self) -> float:
        return 20.0


In [ ]:
#hardware

with WebCamera(mode="HgAr", n_lines=64, processing_lvl = 0) as cam:
    cam.collect()
    fig = cam.show(hist_eq=True)
    
fig

In [ ]:
#hardware
plt.subplots(figsize=(12,4))
plt.imshow(cam.dc.data[:,1,:],cmap="gray",
           extent=[cam.settings["index2wavelength_range"][0],cam.settings["index2wavelength_range"][1],0,cam.dc.size[0]],
          aspect=0.3)

## XIMEA Camera

Used for the OpenHSI Camera Mark I. 

Make sure you install the Ximea API beforehand in the instructions https://www.ximea.com/support/wiki/apis/Python 



In [ ]:
#export

@delegates()
class XimeaCamera(OpenHSI):
        
    """Core functionality for Ximea cameras"""
    # https://www.ximea.com/support/wiki/apis/Python
    def __init__(self, xbinwidth:int = 896, xbinoffset:int = 528, exposure_ms:float = 10, serial_num:str = None, **kwargs):
        """Initialise Camera"""
        
        super().__init__(**kwargs)
        
        try:
            from ximea import xiapi
            self.xiapi=xiapi # make avalaible for later access just in case.
        except ModuleNotFoundError:
            warnings.warn("ModuleNotFoundError: No module named 'ximea'.",stacklevel=2)
        
        self.xicam = self.xiapi.Camera()
        
        self.xicam.open_device_by_SN(serial_num) if serial_num else self.xicam.open_device()

        print(f'Connected to device {self.xicam.get_device_sn()}')

        self.xbinwidth  = xbinwidth
        self.xbinoffset = xbinoffset
        self.exposure   = exposure_ms
        self.gain       = 0

        self.xicam.set_width(self.xbinwidth)
        self.xicam.set_offsetX(self.xbinoffset)
        self.xicam.set_exposure_direct(1000*self.exposure)
        self.xicam.set_gain_direct(self.gain)

        self.xicam.set_imgdataformat("XI_RAW16")
        self.xicam.set_output_bit_depth("XI_BPP_12")
        self.xicam.enable_output_bit_packing()
        self.xicam.disable_aeag()
        
        self.xicam.set_binning_vertical(2)
        self.xicam.set_binning_vertical_mode("XI_BIN_MODE_SUM")

        self.rows, self.cols = self.xicam.get_height(), self.xicam.get_width()
        self.img = xiapi.Image()
        
        #self.load_cam_settings()
        
    def __exit__(self, *args, **kwargs):
        self.xicam.stop_acquisition()
        self.xicam.close_device()

    def start_cam(self):
        self.xicam.start_acquisition()
    
    def stop_cam(self):
        self.xicam.stop_acquisition()
    
    def get_img(self) -> np.ndarray:
        self.xicam.get_image(self.img)
        return self.img.get_image_data_numpy()
    
    def get_temp(self) -> float:
        return self.xicam.get_temp()

In [ ]:
#hardware

def run_ximea():
    with XimeaCamera(n_lines=128, processing_lvl = -1, pkl_path="",json_path='assets/cam_settings_ximea.json') as cam:
        cam.start_cam()
        for i in tqdm(range(cam.n_lines)):
            cam.put(cam.get_img())
        cam.stop_cam()

%prun run_ximea()
    #fig = cam.show(robust=True)
    
#fig

## Lucid Vision Labs Camera

Used for the OpenHSI Camera from Sydney Photonics. 

Make sure you installthe Lucid Vision Labs Arena SDK and python api beforehand. These can be found here https://thinklucid.com/downloads-hub/

Any keyword-value pair arguments must match the those avaliable in settings file. camera specfic ones are:

        binxy: number of pixels to bin in (x,y) direction
        win_resolution: size of area on detector to readout (width, height)
        win_offset: offsets (x,y) from edge of detector for a selective 
        exposure_ms: is the camera exposure time to use
        pixel_format: format of pixels readout sensor, ie Mono8, Mono10, Mono10p, Mono10Packed, Mono12, Mono12p, Mono12Packed, Mono16
        mac_addr: str = "1c:0f:af:01:7b:a0",

In [ ]:
# export

@delegates()
class LucidCamera(OpenHSI):

    """Core functionality for Lucid Vision Lab cameras
        
        Any keyword-value pair arguments must match the those avaliable in settings file. camera specfic ones are:

        binxy: number of pixels to bin in (x,y) direction
        win_resolution: size of area on detector to readout (width, height)
        win_offset: offsets (x,y) from edge of detector for a selective 
        exposure_ms: is the camera exposure time to use
        pixel_format: format of pixels readout sensor, ie Mono8, Mono10, Mono10p, Mono10Packed, Mono12, Mono12p, Mono12Packed, Mono16
        mac_addr: str = "1c:0f:af:01:7b:a0",
    """

    # https://thinklucid.com/downloads-hub/
    def __init__(self, **kwargs):
        """Initialise Camera"""

        super().__init__(**kwargs)

        try:
            from arena_api.system import system as arsys

            self.arsys = arsys  # make avalaible for later access just in case.
        except ModuleNotFoundError:
            warnings.warn(
                "ModuleNotFoundError: No module named 'arena_api'.", stacklevel=2
            )

        #init api and connect to device
        try:
            self.arsys.device_infos
            if self.settings["mac_addr"]:
                device_infos={"mac": self.settings["mac_addr"]} # use specfic camera
            else:
                device_infos={} # or use first camera found
            print("{}".format(device_infos))
            self.device = arsys.create_device()[0]
        except:
            warnings.warn(
                "DeviceNotFoundError: Please connect a lucid vision camera and run again.",
                stacklevel=2)
        # allow api to optimise stream
        tl_stream_nodemap = self.device.tl_stream_nodemap
        tl_stream_nodemap["StreamAutoNegotiatePacketSize"].value = True
        tl_stream_nodemap["StreamPacketResendEnable"].value = True

        # init access to device settings
        self.deviceSettings = self.device.nodemap.get_node([
                "AcquisitionFrameRate",
                "AcquisitionFrameRateEnable",
                "AcquisitionMode",
                "AcquisitionStart",
                "AcquisitionStop",
                "BinningHorizontal",
                "BinningVertical",
                "DevicePower",
                "DeviceTemperature",
                "DeviceUpTime",
                "DeviceUserID",
                "ExposureAuto",
                "ExposureTime",
                "Gain",
                "GammaEnable",
                "Height",
                "OffsetX",
                "OffsetY",
                "PixelFormat",
                "ReverseX",
                "ReverseY",
                "Width",
            ]
        )

        (self.deviceSettings["Height"].value, self.deviceSettings["Width"].value) = (
            (self.settings["win_resolution"][1], self.settings["win_resolution"][0]) if self.settings["win_resolution"][0] else (self.deviceSettings["Height"].max,  self.deviceSettings["Width"].max)
        )

        (self.deviceSettings["OffsetY"].value, self.deviceSettings["OffsetX"].value) = (
            (self.settings["win_offset"][1], self.settings["win_offset"][0]) if self.settings["win_offset"][0] else (self.deviceSettings["OffsetX"].max,  self.deviceSettings["OffsetY"].max)
        )

        self.deviceSettings["ExposureAuto"].value = "Off" # always off as we need to match exposure to calibration data
        self.deviceSettings["ExposureTime"].value = self.settings["exposure_ms"] * 1000.0 # requires time in us float
        self.settings["exposure_ms"] = self.deviceSettings["ExposureTime"].value/1000.00  # exposure time rounds, so storing actual value

        self.deviceSettings["Gain"].value = 0.0 # always 0 as we need to match to calibration data

        self.deviceSettings["PixelFormat"].value = self.settings["pixel_format"]

        self.deviceSettings["BinningHorizontal"].value = self.settings["binxy"][0] # binning is symetric on this sensor, no need to set vertical

        self.rows, self.cols = (
            self.deviceSettings["Height"].value,
            self.deviceSettings["Width"].value,
        )

    def __exit__(self, *args, **kwargs):
        self.device.stop_stream()
        #self.arsys.destroy_device()

    def start_cam(self):
        self.device.start_stream(1)

    def stop_cam(self):
        self.device.stop_stream()

    def get_img(self) -> np.ndarray:
        image_buffer = self.device.get_buffer()
        pdata_as16 = ctypes.cast(image_buffer.pdata, ctypes.POINTER(ctypes.c_ushort))
        nparray_reshaped = np.ctypeslib.as_array(
            pdata_as16, (image_buffer.height, image_buffer.width)
        )
        self.device.requeue_buffer(image_buffer)
        return nparray_reshaped

    def get_temp(self) -> float:
        return self.deviceSettings["DeviceTemperature"].value

In [ ]:
#hardware
with LucidCamera(n_lines=10, processing_lvl = -1, pkl_path="",json_path="assets/cam_settings_lucid.json",binxy=[2, 2],resolution=[540,720]) as cam:
    cam.collect()
    print("Camera temp: {}".format(cam.get_temp()))
    #fig = cam.show(robust=True)

Setting File Override: binxy = [2, 2]
Setting File Override: resolution = [540, 720]
{'mac': '1c:0f:af:01:7b:a0'}


100%|██████████| 10/10 [00:00<00:00, 26.93it/s]


Camera temp: 63.0
